In [1]:
import pandas as pd
import warnings
import re
import numpy as np
import time
import PyPDF2
from PyPDF2 import PdfReader
import tabula
import os
import shutil

In [2]:
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'
warnings.simplefilter(action='ignore', category=FutureWarning)
term_size = os.get_terminal_size()

### Select Scouts

Choose scouts

In [3]:
def clean_scouts():
    scout_assemble = pd.read_csv("Scouts/scouted this week.csv")
    scout_assemble.drop(list(scout_assemble.filter(regex='Unnam')), axis=1, inplace=True)
    scout_assemble["Country"] = scout_assemble["TEA Random"].str.split(" - ").str[-1] # get the country and add to new col
    scout_assemble['TEA Random'] = scout_assemble['TEA Random'].str[4:] # remove "TEA"
    scout_assemble["TEA Random"] = scout_assemble["TEA Random"].str.split(" - ").str[0]
    scout_assemble = scout_assemble.rename(columns={'TEA Random': 'Team'})

    global search_MDS_4_scouts
    search_MDS_4_scouts = scout_assemble["Team"].tolist()

clean_scouts()
search_MDS_4_scouts

['Masvingo Utd',
 'Black Aces',
 'Highlanders',
 'Hwange',
 'Shooting',
 'Lancashire Steel',
 'Caps FC',
 'Eastern Lions',
 'E Lions',
 'Dynamos']

Import previously scouted teams and clean up

In [4]:
def filter_scouts(search,return_scouts): # set function to take two variables (these will be return_scouts and search) >> see search_scouted
    print('\n')
    print('=' * term_size.columns)
    return_scouts.drop(list(return_scouts.filter(regex='Unnamed: 0')), axis=1, inplace=True)
    return_scouts.rename(columns = { "Unnamed: 1" : "Nationality" },inplace=True) # rename column
    print('\033[1m' + search + '\n') # print team name in bold
    display(return_scouts) # show the scouted teams
    print('\n')
    print('=' * term_size.columns)

In [5]:
file = pd.read_csv("Turn Data/Turn Data.csv").Filepath.squeeze() # get file path 
s_PV = 32  # PV threshold

def search_scouted(result):
    try:
        pdf_file = open(file, 'rb')  # Open the PDF file
        pdf_reader = PyPDF2.PdfReader(pdf_file)  # Create a PDF reader object

        for page_number in range(len(pdf_reader.pages)):  # Iterate through each page
            text = pdf_reader.pages[page_number].extract_text()  # Extract text from the page
            
            if result in text:
                page = page_number + 1  # Set the page number
                try:
                    st = pd.DataFrame(tabula.read_pdf(file, pages=page)[0]).replace(np.nan, '')
                except IndexError:
                    continue  # Skip to the next page if there's an IndexError
                
                if st.iloc[:1,:1].squeeze() == "Goalkeepers":  # Ensure it's an MDS team sheet
                    # Your existing processing logic here
                    
                    print(f'\033[1m\n{result}\n on {page}')  # Print the team name
                    
                    # --------- filter and apply ---------- #
                    print('\033[1m\n')
                    print('=' * shutil.get_terminal_size((80, 20)).columns)
                    st.drop(list(st.filter(regex='Unnamed: 0')), axis=1, inplace=True)
                    st.rename(columns={"Unnamed: 1": "Nationality"}, inplace=True)
                    print('\033[1m\n' + result + '\n')
                    print('=' * shutil.get_terminal_size((80, 20)).columns)
                    # print('\n')

                    # --------- strip special characters ---------- #
                    st.iloc[:, 3:16] = st.iloc[:, 3:16].map(lambda x: x.rstrip('*+-')) 

                    # --------- break into groups ---------- #
                    s = st.eq(st.columns)
                    try:
                        s_groups = [g.reset_index(drop=True) for _, g in st[~s.iloc[:, 0]].groupby(s.cumsum()[~s.iloc[:, 0]].iloc[:, 0])]
                        s_gks = s_groups[0].drop(index=0)
                        s_deff = s_groups[1].drop(index=0)
                        s_mid = s_groups[2].drop(index=0)
                        s_att = s_groups[3].drop(index=0)
                    except IndexError:
                        continue  # if an IndexError occurs, skip to the next page

                    # --------- rename cols ---------- #
                    s_deff = s_deff.rename(columns={'Dis': 'Agg', 'Han': 'Tac', 'Ref': 'Jud', 'Crs': 'Vis'})
                    s_mid = s_mid.rename(columns={'Dis': 'Agg', 'Han': 'Tac', 'Ref': 'Jud', 'Crs': 'Vis'})
                    s_att = s_att.rename(columns={'Dis': 'Agg', 'Han': 'Tac', 'Ref': 'Jud', 'Crs': 'Vis'})

                    # --------- split ID and names ---------- #
                    s_gks[['ID', 'Name']] = s_gks['ID Name'].str.split(' ', n=1, expand=True)
                    s_gks.drop(columns="ID Name", axis=1, inplace=True)
                    s_gks = s_gks[["ID", "Name", "Age", "Foot", "Sho", "Mov", "Pas", "Con", "Spe", "Sta", "Hea", "Str", "Dis", "Han", "Ref", "Crs", "OA", "SA", "Nationality", "Price"]]

                    # --------- Calculate PV ---------- #
                    s_gks['GK'] = s_gks[['Dis', 'Han', 'Ref', 'Crs']].astype(int).sum(axis=1, skipna=True)

                    s_deff['CB'] = s_deff[['Hea', 'Str', 'Tac', 'Jud']].astype(int).sum(axis=1, skipna=True)
                    s_deff['FB'] = s_deff[['Spe', 'Sta', 'Agg', 'Tac']].astype(int).sum(axis=1, skipna=True)
                    s_deff['SW'] = s_deff[['Pas', 'Con', 'Tac', 'Jud']].astype(int).sum(axis=1, skipna=True)
                    s_deff['WB'] = s_deff[['Mov', 'Pas', 'Spe', 'Sta']].astype(int).sum(axis=1, skipna=True)

                    s_mid['CM'] = s_mid[['Pas', 'Sta', 'Hea', 'Tac']].astype(int).sum(axis=1, skipna=True)
                    s_mid['AM'] = s_mid[['Str', 'Agg', 'Tac', 'Jud']].astype(int).sum(axis=1, skipna=True)
                    s_mid['PL'] = s_mid[['Pas', 'Con', 'Jud', 'Vis']].astype(int).sum(axis=1, skipna=True)
                    s_mid['FR'] = s_mid[['Sho', 'Mov', 'Pas', 'Con']].astype(int).sum(axis=1, skipna=True)
                    s_mid['WG'] = s_mid[['Pas', 'Con', 'Spe', 'Sta']].astype(int).sum(axis=1, skipna=True)

                    s_att['WF'] = s_att[['Sho', 'Mov', 'Con', 'Spe']].astype(int).sum(axis=1, skipna=True)
                    s_att['IF'] = s_att[['Pas', 'Con', 'Jud', 'Vis']].astype(int).sum(axis=1, skipna=True)
                    s_att['CF'] = s_att[['Sho', 'Mov', 'Str', 'Agg']].astype(int).sum(axis=1, skipna=True)
                    s_att['TM'] = s_att[['Con', 'Hea', 'Str', 'Agg']].astype(int).sum(axis=1, skipna=True)

                    # --------- don't show low PVs ---------- #
                    s_gks['GK'] = s_gks['GK'].astype(int); s_gks['GK'] = np.where(s_gks['GK'] < s_PV, "", s_gks['GK'])

                    s_deff['CB'] = s_deff['CB'].astype(int); s_deff['CB'] = np.where(s_deff['CB'] < s_PV, "", s_deff['CB'])
                    s_deff['SW'] = s_deff['SW'].astype(int); s_deff['SW'] = np.where(s_deff['SW'] < s_PV, "", s_deff['SW'])
                    s_deff['WB'] = s_deff['WB'].astype(int); s_deff['WB'] = np.where(s_deff['WB'] < s_PV, "", s_deff['WB'])
                    s_deff['FB'] = s_deff['FB'].astype(int); s_deff['FB'] = np.where(s_deff['FB'] < s_PV, "", s_deff['FB'])

                    s_mid['CM'] = s_mid['CM'].astype(int); s_mid['CM'] = np.where(s_mid['CM'] < s_PV, "", s_mid['CM'])
                    s_mid['AM'] = np.where(s_mid['AM'] < s_PV, "", s_mid['AM'])
                    s_mid['PL'] = np.where(s_mid['PL'] < s_PV, "", s_mid['PL'])
                    s_mid['FR'] = np.where(s_mid['FR'] < s_PV, "", s_mid['FR'])
                    s_mid['WG'] = s_mid['WG'].astype(int); s_mid['WG'] = np.where(s_mid['WG'] < s_PV, "", s_mid['WG'])

                    s_att['WF'] = s_att['WF'].astype(int); s_att['WF'] = np.where(s_att['WF'] < s_PV, "", s_att['WF'])
                    s_att['IF'] = np.where(s_att['IF'] < s_PV, "", s_att['IF'])
                    s_att['TM'] = np.where(s_att['TM'] < s_PV, "", s_att['TM'])
                    s_att['CF'] = np.where(s_att['CF'] < s_PV, "", s_att['CF'])

                    # --------- Show Players by Section ---------- #
                    display("Goalkeepers", s_gks.style.background_gradient(axis=None, subset=['Spe', 'Dis', 'Crs']),
                            "Defenders", s_deff.style.background_gradient(axis=None, subset=['Spe', 'Agg', 'Vis']),
                            "Midfielders", s_mid.style.background_gradient(axis=None, subset=['Spe', 'Agg', 'Vis']),
                            "Attackers", s_att.style.background_gradient(axis=None, subset=['Spe', 'Agg', 'Vis'])
                            )
                    
                else:
                    print(f"{result} is not a valid MDS team sheet.")
                    
            else:
                # If result is not found, try with the last word
                last_word = " "+result.split()[-1]
                if last_word in text:
                    page = page_number + 1  # Set the page number
                    try:
                        st = pd.DataFrame(tabula.read_pdf(file, pages=page)[0]).replace(np.nan, '')
                    except IndexError:
                        continue  # Skip to the next page if there's an IndexError
                    
                    if st.iloc[:1,:1].squeeze() == "Goalkeepers":  # Ensure it's an MDS team sheet
                        # Your existing processing logic here
                        
                        print(f'\033[1m\n{last_word} on page {page}')  # Print the team name
                        
                       
                        # --------- filter and apply ---------- #
                        print('\033[1m\n')
                        print('=' * shutil.get_terminal_size((80, 20)).columns)
                        st.drop(list(st.filter(regex='Unnamed: 0')), axis=1, inplace=True)
                        st.rename(columns={"Unnamed: 1": "Nationality"}, inplace=True)
                        print('\033[1m\n' + result + '\n')
                        print('=' * shutil.get_terminal_size((80, 20)).columns)
                        # print('\n')
    
                        # --------- strip special characters ---------- #
                        st.iloc[:, 3:16] = st.iloc[:, 3:16].map(lambda x: x.rstrip('*+-')) 
    
                        # --------- break into groups ---------- #
                        s = st.eq(st.columns)
                        try:
                            s_groups = [g.reset_index(drop=True) for _, g in st[~s.iloc[:, 0]].groupby(s.cumsum()[~s.iloc[:, 0]].iloc[:, 0])]
                            s_gks = s_groups[0].drop(index=0)
                            s_deff = s_groups[1].drop(index=0)
                            s_mid = s_groups[2].drop(index=0)
                            s_att = s_groups[3].drop(index=0)
                        except IndexError:
                            continue  # if an IndexError occurs, skip to the next page
    
                        # --------- rename cols ---------- #
                        s_deff = s_deff.rename(columns={'Dis': 'Agg', 'Han': 'Tac', 'Ref': 'Jud', 'Crs': 'Vis'})
                        s_mid = s_mid.rename(columns={'Dis': 'Agg', 'Han': 'Tac', 'Ref': 'Jud', 'Crs': 'Vis'})
                        s_att = s_att.rename(columns={'Dis': 'Agg', 'Han': 'Tac', 'Ref': 'Jud', 'Crs': 'Vis'})
    
                        # --------- split ID and names ---------- #
                        s_gks[['ID', 'Name']] = s_gks['ID Name'].str.split(' ', n=1, expand=True)
                        s_gks.drop(columns="ID Name", axis=1, inplace=True)
                        s_gks = s_gks[["ID", "Name", "Age", "Foot", "Sho", "Mov", "Pas", "Con", "Spe", "Sta", "Hea", "Str", "Dis", "Han", "Ref", "Crs", "OA", "SA", "Nationality", "Price"]]
    
                        # --------- Calculate PV ---------- #
                        s_gks['GK'] = s_gks[['Dis', 'Han', 'Ref', 'Crs']].astype(int).sum(axis=1, skipna=True)
    
                        s_deff['CB'] = s_deff[['Hea', 'Str', 'Tac', 'Jud']].astype(int).sum(axis=1, skipna=True)
                        s_deff['FB'] = s_deff[['Spe', 'Sta', 'Agg', 'Tac']].astype(int).sum(axis=1, skipna=True)
                        s_deff['SW'] = s_deff[['Pas', 'Con', 'Tac', 'Jud']].astype(int).sum(axis=1, skipna=True)
                        s_deff['WB'] = s_deff[['Mov', 'Pas', 'Spe', 'Sta']].astype(int).sum(axis=1, skipna=True)
    
                        s_mid['CM'] = s_mid[['Pas', 'Sta', 'Hea', 'Tac']].astype(int).sum(axis=1, skipna=True)
                        s_mid['AM'] = s_mid[['Str', 'Agg', 'Tac', 'Jud']].astype(int).sum(axis=1, skipna=True)
                        s_mid['PL'] = s_mid[['Pas', 'Con', 'Jud', 'Vis']].astype(int).sum(axis=1, skipna=True)
                        s_mid['FR'] = s_mid[['Sho', 'Mov', 'Pas', 'Con']].astype(int).sum(axis=1, skipna=True)
                        s_mid['WG'] = s_mid[['Pas', 'Con', 'Spe', 'Sta']].astype(int).sum(axis=1, skipna=True)
    
                        s_att['WF'] = s_att[['Sho', 'Mov', 'Con', 'Spe']].astype(int).sum(axis=1, skipna=True)
                        s_att['IF'] = s_att[['Pas', 'Con', 'Jud', 'Vis']].astype(int).sum(axis=1, skipna=True)
                        s_att['CF'] = s_att[['Sho', 'Mov', 'Str', 'Agg']].astype(int).sum(axis=1, skipna=True)
                        s_att['TM'] = s_att[['Con', 'Hea', 'Str', 'Agg']].astype(int).sum(axis=1, skipna=True)
    
                        # --------- don't show low PVs ---------- #
                        s_gks['GK'] = s_gks['GK'].astype(int); s_gks['GK'] = np.where(s_gks['GK'] < s_PV, "", s_gks['GK'])
    
                        s_deff['CB'] = s_deff['CB'].astype(int); s_deff['CB'] = np.where(s_deff['CB'] < s_PV, "", s_deff['CB'])
                        s_deff['SW'] = s_deff['SW'].astype(int); s_deff['SW'] = np.where(s_deff['SW'] < s_PV, "", s_deff['SW'])
                        s_deff['WB'] = s_deff['WB'].astype(int); s_deff['WB'] = np.where(s_deff['WB'] < s_PV, "", s_deff['WB'])
                        s_deff['FB'] = s_deff['FB'].astype(int); s_deff['FB'] = np.where(s_deff['FB'] < s_PV, "", s_deff['FB'])
    
                        s_mid['CM'] = s_mid['CM'].astype(int); s_mid['CM'] = np.where(s_mid['CM'] < s_PV, "", s_mid['CM'])
                        s_mid['AM'] = np.where(s_mid['AM'] < s_PV, "", s_mid['AM'])
                        s_mid['PL'] = np.where(s_mid['PL'] < s_PV, "", s_mid['PL'])
                        s_mid['FR'] = np.where(s_mid['FR'] < s_PV, "", s_mid['FR'])
                        s_mid['WG'] = s_mid['WG'].astype(int); s_mid['WG'] = np.where(s_mid['WG'] < s_PV, "", s_mid['WG'])
    
                        s_att['WF'] = s_att['WF'].astype(int); s_att['WF'] = np.where(s_att['WF'] < s_PV, "", s_att['WF'])
                        s_att['IF'] = np.where(s_att['IF'] < s_PV, "", s_att['IF'])
                        s_att['TM'] = np.where(s_att['TM'] < s_PV, "", s_att['TM'])
                        s_att['CF'] = np.where(s_att['CF'] < s_PV, "", s_att['CF'])
    
                        # --------- Show Players by Section ---------- #
                        display("Goalkeepers", s_gks.style.background_gradient(axis=None, subset=['Spe', 'Dis', 'Crs']),
                                "Defenders", s_deff.style.background_gradient(axis=None, subset=['Spe', 'Agg', 'Vis']),
                                "Midfielders", s_mid.style.background_gradient(axis=None, subset=['Spe', 'Agg', 'Vis']),
                                "Attackers", s_att.style.background_gradient(axis=None, subset=['Spe', 'Agg', 'Vis'])
                                )
                        
                    else:
                        pass #print(f"{last_word} is not a valid MDS team sheet.")
                else:
                    pass #print(f"Could not find '{result}' or '{last_word}' in the PDF.")
    
    except Exception as e:
        print(f"Exception occurred while processing {result}: {str(e)}")
        # Optionally, handle the exception as needed

# Iterate through each element in search_MDS_4_scouts
for i in search_MDS_4_scouts:
    search_scouted(i)  # Call the search_scouted function for each element



 FC on page 48



Caps FC



'Goalkeepers'

,ID,Name,Age,Foot,Sho,Mov,Pas,Con,Spe,Sta,Hea,Str,Dis,Han,Ref,Crs,OA,SA,Nationality,Price,GK
1,MAPH,A Muric,25,R,8,7,8,7,6,7,7,8,9,9,9,8,93,Hei,Burnley,-,35
2,DCGH,S Dimitrievski,29,R,7,6,7,6,5,7,7,8,9,9,8,8,87,Exp,Rayo,"£2,023,269",34
3,ILSM,D Iliev,28,R,5,6,6,7,8,6,7,7,9,8,8,8,85,Hei,HJK Helsink,"£1,400,000",33


'Defenders'

,ID Name,Age,Foot,Sho,Mov,Pas,Con,Spe,Sta,Hea,Str,Agg,Tac,Jud,Vis,OA,SA,Nationality,Price,CB,FB,SW,WB
1,RPBH S Ristovski,31,R,8,6,8,7,9,9,8,8,8,9,7,6,93,Exp,D Zagreb,"£3,500,000",32,35,,32
2,RXZL A Rrahmani,31,B,9,7,8,8,5,6,8,10,7,9,8,7,92,Ego/Hei,FK Milano,"£3,438,220",35,,33,
3,JEVP P Jovcev,22,R,5,7,8,6,9,8,9,9,6,7,8,9,91,Inx,Elche,-,33,,,32
4,ZBZB L Zaharievski,24,R,4,9,8,6,9,7,8,7,9,9,7,7,90,,Partizani,"£280,000",,34,,33
5,LLIV E Letaj,26,L,8,8,9,6,9,9,4,8,7,8,7,6,89,,Hajduk Split,"£2,700,000",,33,,35
6,IYQE E Ivanovic (Inj),21,R,5,9,6,7,9,7,7,8,9,8,7,6,88,Inx,Elche,-,,33,,
7,ZHBI R Zejnullahu,20,L,6,6,9,6,9,8,5,7,8,10,6,8,88,,E.Cluj,"£6,000,000",,35,,32
8,HKXF J Haliti,27,R,5,7,7,5,8,7,8,9,6,8,9,8,87,,AIK,"£2,500,000",34,,,


'Midfielders'

,ID Name,Age,Foot,Sho,Mov,Pas,Con,Spe,Sta,Hea,Str,Agg,Tac,Jud,Vis,OA,SA,Nationality,Price,CM,AM,PL,FR,WG
1,IPBG B Idrizi,25,R,9,7,10,9,8,6,7,8,9,8,7,7,95,,FC Schalke,-,,32,33,35,33
2,BOAT E Bardhi,27,R,10,8,9,8,6,8,6,8,9,9,8,5,94,,Trabzonspor,"£13,500,000",32,34,,35,
3,AWNJ E Alioski,31,L,9,8,8,9,8,10,5,6,10,6,7,7,93,Ver/Tea,St Etienne,-,,,,34,35
4,ETEP E Elmas,23,R,8,8,9,9,7,9,8,6,7,6,8,8,93,,Leipzig,-,32,,34,34,34
5,DFDP L Demir,19,R,9,8,9,9,7,7,7,7,7,8,6,8,92,Inj/Inx,Borec MHK,Free,,,32,35,32
6,AKUI A Ademi,32,L,10,5,8,8,6,6,7,10,9,9,8,5,91,Det/Exp,Salzburg,-,,36,,,
7,OURV T Obiekula,24,R,8,6,6,5,10,10,6,9,7,10,4,8,89,Inx,St Etienne,-,32,,,,


'Attackers'

,ID Name,Age,Foot,Sho,Mov,Pas,Con,Spe,Sta,Hea,Str,Agg,Tac,Jud,Vis,OA,SA,Nationality,Price,WF,IF,CF,TM
1,RXCK M Rashica,27,R,10,8,8,8,9,6,7,7,7,8,9,7,94,,Besiktas,"£14,300,000",35,32,32,
2,ZDHE E Zhegrova,24,L,9,8,8,8,9,9,6,8,7,9,7,6,94,Mid,Lille,-,34,,32,
3,MKGR B Miovski,24,L,8,7,7,8,8,9,8,9,9,6,5,8,92,Det,Villarreal,-,,,33,34
4,RULF A Rrahmani,23,B,9,9,6,8,8,9,5,8,9,7,7,6,91,,Steaua,"£13,000,000",34,,35,
